有三种计算图的构建方式：静态计算图、动态计算图以及Autograph。   
Tensorflow 2.0 主要使用的是动态计算图和Autograph。  
动态计算图便于调试，编码效率高，但是执行效率偏低，因为一切都要等到运行的同时才能确定。  

静态计算图执行效率很高，但调试较难。   

而Autograph机制可以将动态图转换成静态计算图，兼收执行效率和编码效率之利。  

使用Autograph机制能够转换得到代码并不是没有任何约束的，有一些编码规范需要遵循。

本章介绍Autograph的编码规范和Autograph转换成静态图的原理。

并介绍使用tf.Module来更好的构建Autograph。  

本篇我们介绍使用Autograph的编码规范。

## 一、Autograph的编码规范总结

- 被@tf.function修饰的函数应尽可能使用TensorFlow中的函数而不是Python中的其他函数。例如使用tf.print而不是print，使用tf.range而不是range，使用tf.constant(True)而不是True.
- 避免在@tf.function修饰的函数内部定义tf.Variable.
- 被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等数据结构变量

## 二、Autograph编码规范解析
#### 1、被@tf.function修饰的函数应尽量使用TensorFlow中的函数而不是Python中的原生函数。

In [9]:
import tensorflow as tf
import numpy as np

@tf.function
def np_random():
    a = np.random.randn(3,3)
    tf.print(a)
@tf.function()
def tf_random():
    a = tf.random.normal((3,3))
    tf.print(a)

In [10]:
np_random()

array([[ 0.47056181,  0.3973242 , -0.28865518],
       [-2.57133791,  1.95241404,  0.37754769],
       [-0.59428166, -0.95681525, -0.94873014]])


In [11]:
np_random()#np_random每次执行都是一样的结果

array([[ 0.47056181,  0.3973242 , -0.28865518],
       [-2.57133791,  1.95241404,  0.37754769],
       [-0.59428166, -0.95681525, -0.94873014]])


In [12]:
tf_random()

[[1.67122173 -0.439273477 -0.758898735]
 [0.301422119 -0.242566556 0.559788883]
 [1.01128662 -0.000271171884 1.529374]]


In [14]:
tf_random()#每次执行都会有重新生成随机数

[[0.809071362 -0.325012356 -0.827406466]
 [1.39463699 0.000627074507 1.54180455]
 [-1.36532164 -2.05275416 -2.61302304]]


#### 2、避免在@tf.function修饰的函数内部定义tf.Variable. 

In [16]:
x = tf.Variable(1.0,dtype=tf.float32)
@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return x
outer_var()

2


<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [18]:
@tf.function
def inner_var():
    x = tf.Variable(1.0,dtype=tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return x
#执行报错
inner_var()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


ValueError: in converted code:

    <ipython-input-18-b2b834dc2b9b>:3 inner_var  *
        x = tf.Variable(1.0,dtype=tf.float32)
    /home/tangxi.zq/.conda/envs/tf2/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:260 __call__
        return cls._variable_v2_call(*args, **kwargs)
    /home/tangxi.zq/.conda/envs/tf2/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:254 _variable_v2_call
        shape=shape)
    /home/tangxi.zq/.conda/envs/tf2/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:65 getter
        return captured_getter(captured_previous, **kwargs)
    /home/tangxi.zq/.conda/envs/tf2/lib/python3.6/site-packages/tensorflow_core/python/eager/def_function.py:502 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.


#### 3、被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等结构类型变量。

In [25]:
tensor_list = []

#@tf.function #加上这一行切换成Autograph结果将不符合预期！！！
@tf.function
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]


## 三、Autograph的机制原理
当我们使用@tf.function装饰一个函数的时候，后面到底发生了什么呢？

例如我们写如下代码。

In [26]:
import tensorflow as tf
import numpy as np

@tf.function(autograph=True)
def myadd(a,b):
    for i in tf.range(3):
        tf.print(i)
    c = a+b
    print("tracing")
    return c

In [27]:
myadd(tf.constant("hello"),tf.constant("world"))

tracing
0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'helloworld'>

发生了2件事情，

第一件事情是创建计算图。

即创建一个静态计算图，跟踪执行一遍函数体中的Python代码，确定各个变量的Tensor类型，并根据执行顺序将算子添加到计算图中。 在这个过程中，如果开启了autograph=True(默认开启),会将Python控制流转换成TensorFlow图内控制流。 主要是将if语句转换成 tf.cond算子表达，将while和for循环语句转换成tf.while_loop算子表达，并在必要的时候添加 tf.control_dependencies指定执行顺序依赖关系。

相当于在 tensorflow1.0执行了类似下面的语句

In [28]:
g = tf.Graph()
with g.as_default():
    a = tf.placeholder(shape=[],dtype=tf.string)
    b = tf.placeholder(shape=[],dtype=tf.string)
    cond = lambda i :i<tf.constant(3)
    def body(i):
        tf.print(i)
        return(i+1)
    loop = tf.while_loop(cond,body,loop_vars=[0])
    loop
    with tf.control_dependencies(loop):
        c = tf.strings.join([a,b])
    print("tracing")

AttributeError: module 'tensorflow' has no attribute 'placeholder'

第二件事情是执行计算图。

相当于在 tensorflow1.0中执行了下面的语句：

In [29]:
with tf.Session(graph=g) as sess:
    sess.run(c,feed_dict={a:tf.constant("hello"),b:tf.constant("world")})

AttributeError: module 'tensorflow' has no attribute 'Session'

因此我们先看到的是第一个步骤的结果：即Python调用标准输出流打印"tracing"语句。

然后看到第二个步骤的结果：TensorFlow调用标准输出流打印1,2,3。

当我们再次用相同的输入参数类型调用这个被@tf.function装饰的函数时，后面到底发生了什么？

例如我们写下如下代码。

In [30]:
myadd(tf.constant("good"),tf.constant("morning"))

0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'goodmorning'>

只会发生一件事情，那就是上面步骤的第二步，执行计算图。

所以这一次我们没有看到打印"tracing"的结果。

当我们再次用不同的的输入参数类型调用这个被@tf.function装饰的函数时，后面到底发生了什么？

例如我们写下如下代码。

In [31]:
myadd(tf.constant(1),tf.constant(2))

tracing
0
1
2


<tf.Tensor: shape=(), dtype=int32, numpy=3>

由于输入参数的类型已经发生变化，已经创建的计算图不能够再次使用。

需要重新做2件事情：创建新的计算图、执行计算图。

所以我们又会先看到的是第一个步骤的结果：即Python调用标准输出流打印"tracing"语句。

然后再看到第二个步骤的结果：TensorFlow调用标准输出流打印1,2,3。

需要注意的是，如果调用被@tf.function装饰的函数时输入的参数不是Tensor类型，则每次都会重新创建计算图。

例如我们写下如下代码。两次都会重新创建计算图。因此，一般建议调用@tf.function时应传入Tensor类型。

In [32]:
myadd("hello","world")
myadd("good","morning")

tracing
0
1
2
tracing
0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'goodmorning'>

## 四、重新理解Autograph的编码规范

In [33]:
了解了以上Autograph的机制原理，我们也就能够理解Autograph编码规范的3条建议了。

- 1、被@tf.function修饰的函数应尽量使用TensorFlow中的函数而不是Python中的其他函数。例如使用tf.print而不是print.

解释：Python中的函数仅仅会在跟踪执行函数以创建静态图的阶段使用，普通Python函数是无法嵌入到静态计算图中的，所以 在计算图构建好之后再次调用的时候，这些Python函数并没有被计算，而TensorFlow中的函数则可以嵌入到计算图中。使用普通的Python函数会导致 被@tf.function修饰前【eager执行】和被@tf.function修饰后【静态图执行】的输出不一致。

- 2、 避免在@tf.function修饰的函数内部定义tf.Variable
解释：如果函数内部定义了tf.Variable,那么在【eager执行】时，这种创建tf.Variable的行为在每次函数调用时候都会发生。但是在【静态图执行】时，这种创建tf.Variable的行为只会发生在第一步跟踪Python代码逻辑创建计算图时，这会导致被@tf.function修饰前【eager执行】和被@tf.function修饰后【静态图执行】的输出不一致。实际上，TensorFlow在这种情况下一般会报错。

SyntaxError: invalid character in identifier (<ipython-input-33-b1a6044dbf7f>, line 1)